<a href="https://colab.research.google.com/github/1BM23CS345/AI/blob/main/Lab_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
class Literal:
    def __init__(self, name, negated=False):
        self.name = name
        self.negated = negated

    def __repr__(self):
        return f"¬{self.name}" if self.negated else self.name

    def __eq__(self, other):
        return isinstance(other, Literal) and self.name == other.name and self.negated == other.negated

    def __hash__(self):
        return hash((self.name, self.negated))
print('Suhas B P (1BM23CS345)')
def convert_to_cnf(sentence):
    return sentence  # Placeholder for CNF conversion logic

def negate_literal(literal):
    return Literal(literal.name, not literal.negated)

def resolve(clause1, clause2):
    resolvents = []
    for literal1 in clause1:
        for literal2 in clause2:
            if literal1.name == literal2.name and literal1.negated != literal2.negated:
                new_clause = (set(clause1) - {literal1}) | (set(clause2) - {literal2})
                resolvents.append(frozenset(new_clause))
    return resolvents

def prove_conclusion(premises, conclusion):
    cnf_premises = [convert_to_cnf(p) for p in premises]
    cnf_conclusion = convert_to_cnf(conclusion)
    negated_conclusion = frozenset({negate_literal(lit) for lit in cnf_conclusion})

    clauses = set(frozenset(p) for p in cnf_premises)
    clause_list = list(clauses)
    id_map = {}
    parents = {}
    clause_id = 1


    for c in clause_list:
        id_map[c] = f"C{clause_id}"
        clause_id += 1

    id_map[negated_conclusion] = f"C{clause_id} (¬Conclusion)"
    clauses.add(negated_conclusion)
    clause_list.append(negated_conclusion)
    clause_id += 1

    print("\n--- Initial Clauses ---")
    for c in clause_list:
        print(f"{id_map[c]}: {set(c)}")

    print("\n--- Resolution Steps ---")

    while True:
        new_clauses = set()
        for i in range(len(clause_list)):
            for j in range(i + 1, len(clause_list)):
                resolvents = resolve(clause_list[i], clause_list[j])
                for r in resolvents:
                    if r not in id_map:
                        id_map[r] = f"C{clause_id}"
                        parents[r] = (id_map[clause_list[i]], id_map[clause_list[j]])
                        clause_id += 1

                    print(f"{id_map[r]} = RESOLVE({id_map[clause_list[i]]}, {id_map[clause_list[j]]}) -> {set(r)}")

                    if not r:
                        print(f"\nEmpty clause derived! ({id_map[r]})")
                        print("\n--- Resolution Tree ---")
                        print_resolution_tree(parents, id_map, r)
                        return True

                    new_clauses.add(r)

        if new_clauses.issubset(clauses):
            print("\nNo new clauses can be derived.")
            return False

        clauses |= new_clauses
        clause_list = list(clauses)

def print_resolution_tree(parents, id_map, empty_clause):
    """Recursively print resolution tree leading to the empty clause."""
    def recurse(clause):
        if clause not in parents:
            print(f" {id_map[clause]}: {set(clause)}")
            return
        left, right = parents[clause]
        print(f" {id_map[clause]} derived from {left} and {right}")
        for parent_clause, parent_name in zip([k for k, v in id_map.items() if v in [left, right]], [left, right]):
            recurse(parent_clause)

def parse_literal(lit_str):
    lit_str = lit_str.strip()
    if lit_str.startswith("¬") or lit_str.startswith("~"):
        return Literal(lit_str[1:], True)
    return Literal(lit_str, False)

def get_user_input():
    premises = []
    num_premises = int(input("Enter number of premises: "))
    for i in range(num_premises):
        clause_str = input(f"Enter clause {i+1} (e.g., A, ¬B, C): ")
        literals = {parse_literal(l) for l in clause_str.split(",")}
        premises.append(literals)

    conclusion_str = input("Enter conclusion (e.g., C or ¬C): ")
    conclusion = {parse_literal(l) for l in conclusion_str.split(",")}
    return premises, conclusion

if __name__ == "__main__":
    premises, conclusion = get_user_input()

    if prove_conclusion(premises, conclusion):
        print("\n Conclusion can be proven from the premises.")
    else:
        print("\n Conclusion cannot be proven from the premises.")



Suhas B P (1BM23CS345)
Enter number of premises: 4
Enter clause 1 (e.g., A, ¬B, C): A
Enter clause 2 (e.g., A, ¬B, C): -A
Enter clause 3 (e.g., A, ¬B, C): -B
Enter clause 4 (e.g., A, ¬B, C): c
Enter conclusion (e.g., C or ¬C): c

--- Initial Clauses ---
C1: {A}
C2: {-B}
C3: {c}
C4: {-A}
C5 (¬Conclusion): {¬c}

--- Resolution Steps ---
C6 = RESOLVE(C3, C5 (¬Conclusion)) -> set()

Empty clause derived! (C6)

--- Resolution Tree ---

 Conclusion can be proven from the premises.
